# Methylation analysis
from https://labs.epi2me.io/notebooks/Modified_Base_Tutorial.html


**It is necessary to merge the bam files first and make a BED file of the modified bases.** 

To do so, copy the bam files to iPOP-UP server and run `modbam2bed.sh`. You obtain a file named `SAMPLE_Merged.cpg.bed`. Copy this file to IFB cluster before running the notebook.  

**Be sure to choose the kernel `nanopore_analysis` (upper right) to run this notebook.**

## Summarising the data

In [ ]:
# Modified base summary parsing (click play)
import pandas as pd
bedmethyl = "PATHto/SAMPLE_Merged.cpg.bed"   ### in principle the only line to adapt to your run. 
methdata = pd.read_csv(
    bedmethyl, sep='\t',
    header=None,
    names=["chrom", "start", "end", "name", "score", "strand", "tstart", "tend", "color", "coverage", "freq", "canon", "mod", "filt"])
methdata.head()

In [ ]:
# Coverage plot code (click play)
import aplanat
from aplanat import hist

names = ('fwd', 'rev')
fwdmeth = methdata.loc[methdata['strand'] == "+"]
revmeth = methdata.loc[methdata['strand'] == "-"]
plot = hist.histogram(
    [x["coverage"] for x in (fwdmeth, revmeth)], names=names,
    colors=['maroon', 'darkolivegreen'],
    binwidth=1, style='line', title='Coverage distribution',
    xlim=(0,20))
plot.xaxis.axis_label = 'coverage'
plot.yaxis.axis_label = 'frequency'
aplanat.show(plot, background='#F4F4F4')

In [ ]:
# Methylation summary plot code (click play)
coverage_mask =  10

from bokeh.layouts import gridplot
from aplanat import annot

# join the reverse to the fwd assuming sites are one apart
# NOTE: we could just use the aggregated results from modbam2bed
print("Joining forward and reverse strand joins assuming 1-base offset.")
tmp = revmeth.copy()
tmp['start'] -= 1
tmp['end'] -= 1
methjoin = pd.merge(fwdmeth, tmp, on=("chrom", "start"), suffixes=(".fwd", ".rev"))
methjoin["coverage"] = methjoin["coverage.fwd"] + methjoin["coverage.rev"]

# proportion by site (modified by MH)
p1 = hist.histogram(
    [methdata['freq'].dropna()],
    colors=['steelblue'], xlim=(-5, 105),
    bins=20, title='Methylation proportion by site')
p1.xaxis.axis_label = 'methylation proportion'
p1.yaxis.axis_label = 'frequency'

# strand bias - remove the trivial case
bias = methjoin['freq.fwd'] - methjoin['freq.rev']
bias = bias.loc[(bias<50) & (bias>-50)]
p2 = hist.histogram(
    [bias], colors=['steelblue'], bins=20,
    title="Methylation strand bias by site.")
p2.xaxis.axis_label = '(fwd. meth. prop.) - (rev. meth. prop.)'
p2.yaxis.axis_label = 'frequency'

aplanat.show(gridplot([[p1, p2]]) , background='#F4F4F4')

In [ ]:
# Coverage plot code
from aplanat import points
import ipywidgets as widgets
from epi2melabs.notebook import InputForm, InputSpec


def plot_callback(inputs):
    try:
        chrom, coords = inputs.region.split(":")
        start, stop = (int(x) for x in coords.split("-"))
    except Exception as e:
        print('Cannot parse region as "chrom:start-stop".')

    # filter data by inputs
    select = (
        (methjoin['coverage'] >= inputs.coverage_mask) &
        (methjoin['chrom'] == chrom) & 
        (methjoin['start'] > start) & 
        (methjoin['start'] < stop))
    d = methjoin.loc[select]

    def down_sample(df, plot_limit=350000):
        # if we have a lot of points, remove some to avoid bokeh dying.
        if len(df) > plot_limit:
            display("Warning: Downsampling points to {} entries. Select a smaller region to show all points.".format(plot_limit))
            df = df.sample(n=plot_limit)
        else:
            display("Showing all sites.")
        return df

    # create a plot
    title = '{} coverage'.format(inputs.region)

    if inputs.colour_by == "orientation":
        d = down_sample(d, plot_limit=350000)
        xs = [d['start']] * 2
        ys = [d['coverage.fwd'], -d['coverage.rev']]
        colors=['maroon', 'orange']
        names=['fwd', 'rev']
    elif inputs.colour_by == "mod. status":
        d = down_sample(d, plot_limit=350000)
        xs = [d['start']] * 2
        ys = [
            +d['mod.fwd'] + d['mod.rev'],
            -d['canon.rev'] - d['canon.rev']]
        colors = ['blue', 'green']
        names = ['modified', 'canonical']
    elif inputs.colour_by == "both":
        d = down_sample(d, plot_limit=125000)
        xs = [d['start']] * 4
        title += ' Positive: fwd strand, Negative: rev strand'
        ys = [
            +d['mod.fwd'], +d['canon.fwd'],
            -d['mod.rev'], -d['canon.fwd']]
        colors = ['blue', 'green', 'blue', 'green']
        names = ['modified', 'canonical', None, None] 
    else:
        raise ValueError("Unrecognised 'colour_by'.")

    plot = points.points(
        xs, ys, colors=colors, names=names, height=300, width=1200, ylim=(-15,15))
    plot.xaxis.formatter.use_scientific = False
    plot.xaxis.axis_label = 'position'
    plot.yaxis.axis_label = 'frequency'
    aplanat.show(plot, background='#F4F4F4')


plot_form = InputForm(
    InputSpec("region", "Region", "chr1:5000000-10000000"),
    InputSpec("colour_by", "Colour by",["mod. status", "orientation", "both"]),
    InputSpec("coverage_mask", "Coverage mask", widgets.IntText(10)))
plot_form.add_process_button(plot_callback)
plot_form.display()